# Seismic Data Processing & Cabled Arrays 

![cabled_array](./FIGURES/cabled_array.jpg)

The following notebook is intended to give you an overview of the key methods used to **download**, **process** and **view** seismic data. Hopefully, at the end of this notebook, you'll be able to play a little bit around with your own data !

The functions and modules used in this notebook are largely inspired from those developped in the [**ObsPy**](https://github.com/obspy/obspy/wiki) package. ObsPy offers a broad range of [functions](https://docs.obspy.org/packages/index.html) whose aim is to standardize and ease the maniuplation of seismic data.

In this notebook we'll focus our study on the **OOI** (and **ONC**...) cabled arrays



## Yaml ! Yaml! Install the correct conda environment

Not everybody has the same python packages installed on its own computer, but to run this notebook we need some specific packages, that can be grouped into one **conda environment**. So, if is not already done, install [conda](https://www.anaconda.com/download/#linux). 

When it's done, let's create an environment `myenv` from the following file [SeismicDataProcessing.yml](./SeismicDataProcessing.yml). Copy paste the following command line in your terminal:  
`conda env create -n myenv -f SeismicDataProcessing.yml`

That might take some time! No worry!

Then activate the environment `myenv`. Copy paste the following command line in your terminal:  
`source activate myenv`

You can now run the notebook :)

## Let's import some useful packages !

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import copy
import pickle
from datetime import datetime
from obspy import UTCDateTime
import sys

In [ ]:
print(sys.path)

## Accessing the network metadata

The **metadata** contains all useful details needed to describe the network (number of stations and coordinates, type of channels...). The **metadata** is different from the **waveform data** itself but they are complementary if we want to process correctly the waveforms.

Once you start a new research subject related to seismology, the first thing you need is the code (i.e. "AZ" or "BB") related to your network. A large number of networks are listed on [**IRIS**](http://ds.iris.edu/mda).

Here, we'll use the networks codes ["OO"](http://ds.iris.edu/mda/OO)(Ocean Observatories Initiative) and ["NV"](http://ds.iris.edu/mda/NV)(formerly Neptune Canada and now Ocean Networks Canada).

Let's see how to get the metadata associated to our network.

First we need to initialize a `Client` object which will help us retrieve the data by redirecting to the appropriate webservice. The argument given to `Client` can be the webservice name or it's URL. 

In [ ]:
from obspy.clients.fdsn import Client

client_name='IRIS'
client_name='http://service.iris.edu'

client=Client(client_name)

print(client)

See below the list of pre-registered webservices in ObsPy:

In [ ]:
from obspy.clients.fdsn.header import URL_MAPPINGS

for key in sorted(URL_MAPPINGS.keys()):
    print("{0:<7} {1}".format(key,  URL_MAPPINGS[key]))

Now that the `Client` has been loaded we can retrieve our network info:

In [ ]:
### Parameters

#net_code='NV'
net_code='NV,OO'

starttime = UTCDateTime("2015-01-01")
endtime = UTCDateTime("2015-01-02")

### Read network data

inventory = client.get_stations(network=net_code,
                                starttime=starttime,
                                endtime=endtime)

print(inventory)

### Plot inventory
plt.ioff()
inventory.plot(color_per_network=True) ### Plot global
inventory.plot(projection='local',color_per_network=True) ### Plot local
plt.show()

It is possible to be more specific when requesting the data, for example by choosing only the vertical channels (i.e. Z). Wildcards can be used to specify `network`,`station` or `channel` parameters. 

> **Tip**: To get more info you can also specify the `level`. When `level="response"` is set, it means we will retrieve the instrument response parameters which are indispensable to correct our waveforms, we'll see that later ;)

In [ ]:
inventory = client.get_stations(network='OO',station='AXAS1,AXAS2',channel='*Z',
                                starttime=starttime,
                                endtime=endtime,level="channel")
print(inventory)


After loading the `Inventory` object it's also possible to select some specific stations afterwards:

In [ ]:
inventory_select = inventory.select(channel="*Z", station="AXAS1")
print(inventory_select)

### Instrument Response

Amplitudes observed on seismograms do not relate directly to the true ground motion. The true motion recorded is actually a combination of source characteristics, raypath attenuation and instrument filtering. Thus the amplitude read on a seismogram can be represented as the following convolution:

$u(t)= s(t)*r(t)*i(t)$

where $s$ depicts the source , $r$ is the attenuation and $i$ is the instrument filter.
In the frequency domain, convolution becomes a simple multiplication. Thus by applying a Fourier transform to the seismogram, dividing by the instrument response $I(\omega)$, and going back to the time domain, we can remove the effect of the instrument on the particle motion.

Let's plot the instrument response for a specific channel:

In [ ]:
inventory = client.get_stations(network='OO',station='AXAS1',channel='EHZ',
                                starttime=starttime,
                                endtime=endtime,level="response")

inventory.plot_response(min_freq=0.001)
plt.show()

> We'll see later how to remove the  instrument response, be patient ;)

### Station inventory I/O

The inventory can also be written to a file, so that it can be stored locally on your computer for later use:

In [ ]:
inventory = client.get_stations(network='OO',station='AXAS1',channel='*',
                                starttime=starttime,
                                endtime=endtime,level="response")

#help(inventory.write) # to check available format
inventory.write('station.xml',format='STATIONXML')


> **Tip**: You can check what's inside your `.xml` file by dragging and dropping it directly to your favorite webbrowser. Please take some time to do it as it contains some useful info.

> <font color=blue>**Info**</font>: [Station XML](https://www.fdsn.org/xml/station/) files have become the new standard to handle station metadata, it's much more comprehensible than dataless SEED files.

## Let's get some waveforms!

So far we have seen how to get the metadata associated to a network, it's now time to see what mother earth has to give!

### Reading seismograms

The method is pretty similar than the one used for getting stations metadata, except that here we'll use the `.get_waveforms` method instead of `.get_stations` (hard to make it simpler !). The output is then redirected to a `Stream` object instead of an `Inventory` object.

In [ ]:
starttime = UTCDateTime("2015-01-22T10:34:21")
duration = 20

st = client.get_waveforms(network='OO',station='AX*',location="*",channel='E*Z',
                                starttime=starttime,
                                endtime=starttime+duration)

print(type(st))
print(st)
print(st.print_gaps())

A `Stream` object can have multiple `Trace` objects. To access some info about the trace you can do:

In [ ]:
trace=st[0]
print(type(trace))
print(trace.stats)
print(trace.times())
print(trace.data)

### Plotting seismograms

You can choose to plot the data by yourself by getting `trace.data` and `trace.times()` but ObsPy offers convenient methods to plot seismograms without directly dealing with the data.

* Classic plots :

In [ ]:
st.plot()

* Day plots:

In [ ]:
starttime = UTCDateTime("2015-01-22T00:00:00")

st = client.get_waveforms(network='OO',station='AXAS2',location="*",channel='E*Z',
                                starttime=starttime,
                                endtime=starttime+86400)

trace=st[0]
trace.plot(type='dayplot',color=['k', 'r', 'b', 'g'],interval=60) ## interval is in minutes and can be changed

> We can see that the above trace has missing values, let's see how we can handle gaps!

### Handling gaps

With the `Stream.merge` method we can fill gaps with user defined values. The method will merge traces that have same ids (i.e. same station + loc + channel code). 

In [ ]:
st_filled_gaps=None
st_filled_gaps=copy.deepcopy(st)
#print(st_filled_gaps[0].stats)
#print(st_filled_gaps[1].stats)
st_filled_gaps[0].stats.sampling_rate=200  # allow merging
st_filled_gaps.merge(method=0, fill_value=0)

trace=st_filled_gaps[0]
trace.plot(type='dayplot',color=['k', 'r', 'b', 'g'],interval=60) ## interval is in minutes and can be changed


Using ObsPy utilities to load data and detect gaps it's possible to plot a figure showing the data coverage for a specific network and for a defined time range, see example below for network ONC (~7 years):

![ONC](./FIGURES/ONC.png)

### Processing seismograms

#### Filtering

As you can see from the plot above, seimic onsets generated by earthquakes are sometimes hard to see on raw seismograms, this is particularly true in noisy environments and for low magnitudes earthquakes. It's generally a good idea to filter the seismograms to enhance the signal associated to an earthquake, let's take a single trace to see that.

In [ ]:
starttime = UTCDateTime("2015-01-22T10:34:25")
duration = 20

st = client.get_waveforms(network='OO',station='AXAS1',location="*",channel='E*Z',
                                starttime=starttime,
                                endtime=starttime+duration)

st.merge()
trace=st[0]

> **Info**: Generally one should take care when choosing the higher corner frequency as it shouldn't exceed the Nyquist frequency, here, ObsPy will take care of that by sending a warning if necessary!

In [ ]:
trace.plot()

#### Apply different to the trace

trace_filter=copy.copy(trace)
trace_filter.filter('bandpass', freqmin=5.0, freqmax=50,corners=3, zerophase=True)
trace_filter.plot()

trace_filter=copy.copy(trace)
trace_filter.filter('highpass', freq=50.0, corners=2, zerophase=True)
trace_filter.plot()

trace_filter=copy.copy(trace)
trace_filter.filter('lowpass', freq=5.0, corners=2, zerophase=True)
trace_filter.plot()

#### Spectrograms

In [ ]:
starttime = UTCDateTime("2015-01-22T00:00:00")
duration = 60*10 # Let's take 10 minutes

st = client.get_waveforms(network='OO',station='AXAS1',location="*",channel='E*Z',
                                starttime=starttime,
                                endtime=starttime+duration)

st.merge()
trace=st[0]

trace_filter=copy.copy(trace)
trace_filter.filter('bandpass', freqmin=5.0, freqmax=50,corners=3, zerophase=True)

trace_filter.plot()

#fig,ax=plt.subplots()

cmap= matplotlib.cm.get_cmap('jet')

trace_filter.spectrogram(cmap=cmap,wlen=20,per_lap=0.9)


#### Remove instrument response

To remove the instrument response we need to load the `Stream` object with the response attached to it:

In [ ]:
starttime = UTCDateTime("2015-01-22T10:34:00")
duration = 60 # Let's take 10 minutes

st = client.get_waveforms(network='OO',station='AXAS1',location="*",channel='E*Z',
                                starttime=starttime,
                                endtime=starttime+duration,attach_response=True)

trace=st[0]

trace_sensi=trace.copy()
trace_sensi.remove_sensitivity()  # This removes the multiplication factor applied to the seismogram
trace.plot()
trace_sensi.plot()
pre_filt = [0.5, 1, 50, 70]
trace.remove_response(output="DISP",plot=True,water_level=0)

trace_filter=trace.copy()

trace_filter.filter('bandpass', freqmin=5.0, freqmax=50,corners=3)
trace_filter.plot()



### Saving seismograms to file

Once you loaded the seismograms into the `Stream` object, you may want to save them locally to gain some time by avoiding making request to the webservice every time you want to play around with the data. To do so:

In [ ]:
st = client.get_waveforms(network='OO',station='AXAS1',location="*",channel='E*Z',
                                starttime=starttime,
                                endtime=starttime+duration,attach_response=True)
st.write('mytrace.mseed',format='MSEED')

And to load it back just do:

In [ ]:
from obspy import read

new_st=read('mytrace.mseed',format='MSEED')

print(new_st)

### Plotting picks on top of seismograms

When having onset picks from seismological catalog it's possible to plot them on top of waveforms, see example below when using a function derived from [ObsPyck](https://github.com/megies/obspyck):

![Waveforms](./FIGURES/Waveforms.png)

> Figure: OOI network example with P (black) and S (red) onsets 